# 4-3 Rules of Using the AutoGraph

There are three ways of constructing graph: static, dynamic and Autograph.

TensorFlow 2.X uses dynamic graph and Autograph.

Dynamic graph is easier for debugging with higher encoding efficiency, but with lower efficiency in execution.

Static graph has high efficiency in execution, but more difficult for debugging.

Autograph mechanism transforms dynamic graph into static graph, making allowance for both executing and encoding efficiencies.

There are certain rules for the code that is able to converted by Autograph, or it could result in failure or unexpected results.

We are going to introduce the coding rules of Autograph and its mechanism of converting into static graph, together with introduction about how to construct Autograph using `tf.Module`.

This section introduce the coding rules of using Autograph. We will introduce the mechanisms of Autograph in next section and explain the logic behind the rules there.

<!-- #region -->
### 1. Summarization of the Coding Rules of Autograph


* 1. We should use the TensorFlow-defined functions to be decorated by `@tf.function` as much as possible, instead of those Python functions. For instance, `tf.print` should be used instead of `print`; `tf.range` should be used instead of `range`; `tf.constant(True)` should be used instead of `True`.

* 2. Avoid defining `tf.Variable` inside the decorator `@tf.function`.

* 3. Functions that are decorated by `@tf.function` cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.
<!-- #endregion -->

### 2. Explanations to the Autograph Coding Rules


 **2.1  We should use the TensorFlow-defined functions to be decorated by `@tf.function` as much as possible, instead of those Python functions.**


In [1]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)


In [2]:
# Same results after each execution of np_random
np_random()
np_random()

array([[ 0.25134437, -0.03228947, -0.29466093],
       [ 0.54150381,  0.67923698, -0.51601442],
       [ 0.44043714, -0.42121957, -1.00554045]])
array([[ 0.25134437, -0.03228947, -0.29466093],
       [ 0.54150381,  0.67923698, -0.51601442],
       [ 0.44043714, -0.42121957, -1.00554045]])


In [3]:
# New random numbers are generated after each execution of tf_random
tf_random()
tf_random()

[[-1.0011673 -0.0995076299 -2.32573843]
 [1.52956295 -0.982268512 -0.447938532]
 [-0.93382287 0.434479773 -2.08646727]]
[[-0.790998399 0.0259545967 0.0513409264]
 [0.142200559 0.390263647 -0.902663]
 [-1.16874099 0.14255169 0.235685781]]


**2.2 Avoid defining `tf.Variable` inside the decorator `@tf.function`.**

In [4]:
# Avoid defining tf.Variable inside the decorator @tf.function.

x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()


2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [5]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

# Error after execution
#inner_var()
#inner_var()

**2.3  Functions that are decorated by `@tf.function` cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.**


In [6]:
tensor_list = []

#@tf.function # Autograph will result in something unexpected if executing this line
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [7]:
tensor_list = []

@tf.function # Autograph will result in something unexpected if executing this line
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
